# FIPO Focused Learning: Model-Agnostic Optimization

## 🎯 Learning Objectives

Notebook này tập trung vào khả năng **Model-Agnostic** của FIPO - một trong những đóng góp quan trọng nhất:

1. Hiểu sự khác biệt giữa model-specific và model-agnostic optimization
2. Phân tích cách FIPO hoạt động với các downstream generators khác nhau
3. So sánh hiệu quả trên nhiều model sizes và architectures
4. Implement strategies cho cross-model optimization

## 📚 Paper References

- **Section 2.1**: Task Formulation (model-agnostic approach)
- **Section 3.2**: Experimental Results across models
- **Table 2**: Performance on various downstream generators
- **Figure 4**: Comparison across 6 different LLMs

## 1. Understanding Model-Agnostic vs Model-Specific

### 1.1 The Generalization Challenge

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle, FancyBboxPatch
import networkx as nx

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Set random seed
np.random.seed(42)

In [ ]:
def visualize_optimization_approaches():
    """Compare model-specific vs model-agnostic approaches"""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # Left: Model-specific (Ad-hoc APO)
    ax1.set_title("Model-Specific Optimization\n(Traditional APO)", 
                 fontsize=16, weight='bold', pad=20)
    
    # Draw in-box testing loop
    in_box_rect = FancyBboxPatch((0.1, 0.6), 0.8, 0.3, 
                                boxstyle="round,pad=0.05",
                                facecolor='lightcoral', 
                                edgecolor='darkred',
                                linewidth=2)
    ax1.add_patch(in_box_rect)
    ax1.text(0.5, 0.75, "In-box Generator\n(e.g., GPT-3.5)", 
            ha='center', va='center', fontsize=12, weight='bold')
    
    # Draw optimization dependency
    ax1.arrow(0.5, 0.6, 0, -0.15, head_width=0.05, head_length=0.03, 
             fc='black', ec='black')
    ax1.text(0.52, 0.5, "Depends on", ha='left', fontsize=10)
    
    # Optimizer
    opt_rect = FancyBboxPatch((0.2, 0.2), 0.6, 0.2,
                             boxstyle="round,pad=0.05",
                             facecolor='lightblue',
                             edgecolor='darkblue',
                             linewidth=2)
    ax1.add_patch(opt_rect)
    ax1.text(0.5, 0.3, "API Optimizer\n(e.g., GPT-4)", 
            ha='center', va='center', fontsize=12, weight='bold')
    
    # Show poor generalization
    out_models = [(0.05, 0.02, "Model A"), (0.35, 0.02, "Model B"), 
                 (0.65, 0.02, "Model C")]
    for x, y, label in out_models:
        rect = Rectangle((x, y), 0.25, 0.08, 
                        facecolor='lightgray', edgecolor='gray')
        ax1.add_patch(rect)
        ax1.text(x + 0.125, y + 0.04, label, ha='center', va='center', 
                fontsize=9)
        ax1.text(x + 0.125, y - 0.03, "❌", ha='center', fontsize=12)
    
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    ax1.axis('off')
    
    # Right: Model-agnostic (FIPO)
    ax2.set_title("Model-Agnostic Optimization\n(FIPO)", 
                 fontsize=16, weight='bold', pad=20)
    
    # Local optimizer
    local_rect = FancyBboxPatch((0.2, 0.6), 0.6, 0.3,
                               boxstyle="round,pad=0.05",
                               facecolor='lightgreen',
                               edgecolor='darkgreen',
                               linewidth=2)
    ax2.add_patch(local_rect)
    ax2.text(0.5, 0.75, "Local FIPO Optimizer\n(Trained Offline)", 
            ha='center', va='center', fontsize=12, weight='bold')
    
    # No dependency arrow - independent!
    ax2.text(0.5, 0.5, "✨ Independent ✨", ha='center', fontsize=12, 
            style='italic', color='darkgreen')
    
    # Show good generalization
    out_models = [(0.05, 0.3, "Model A"), (0.35, 0.3, "Model B"), 
                 (0.65, 0.3, "Model C"),
                 (0.05, 0.15, "Model D"), (0.35, 0.15, "Model E"), 
                 (0.65, 0.15, "Model F")]
    
    for x, y, label in out_models:
        rect = Rectangle((x, y), 0.25, 0.08, 
                        facecolor='lightgreen', edgecolor='green')
        ax2.add_patch(rect)
        ax2.text(x + 0.125, y + 0.04, label, ha='center', va='center', 
                fontsize=9)
        ax2.text(x + 0.125, y - 0.05, "✓", ha='center', fontsize=14, 
                color='green')
    
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 1)
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_optimization_approaches()

### 1.2 Mathematical Formulation Comparison

In [ ]:
def explain_mathematical_difference():
    """Explain mathematical difference between approaches"""
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    
    # Traditional Ad-hoc APO
    ax1.text(0.5, 0.9, "Traditional Ad-hoc APO (Equations 6-7)", 
            transform=ax1.transAxes, ha='center', fontsize=16, weight='bold')
    
    equations_adhoc = [
        r"$\hat{x}_t^{oi+1} = \arg\max_{M_{o-api}} p(\hat{x}_t^{oi+1} | x_t^{oi}, \hat{y}_t^{oi})$",
        r"$\hat{y}_t^{oi} = \arg\max_{M_{g-in}} p(\hat{y}_t^{oi} | x_t^{oi})$",
        "",
        "⚠️ Requires in-box testing response $\hat{y}_t^{oi}$",
        "⚠️ Tied to specific $M_{g-in}$ generator"
    ]
    
    y_pos = 0.7
    for eq in equations_adhoc:
        if eq.startswith("$"):
            ax1.text(0.5, y_pos, eq, transform=ax1.transAxes, ha='center', 
                    fontsize=14, fontfamily='serif')
        elif eq.startswith("⚠️"):
            ax1.text(0.5, y_pos, eq, transform=ax1.transAxes, ha='center', 
                    fontsize=12, color='red')
        y_pos -= 0.12
    
    ax1.axis('off')
    
    # FIPO Approach
    ax2.text(0.5, 0.9, "FIPO Model-Agnostic (Equations 1, 5)", 
            transform=ax2.transAxes, ha='center', fontsize=16, weight='bold')
    
    equations_fipo = [
        "Training:",
        r"$\hat{x}_o = \arg\max_{M_{o-local}} p(\hat{x}_o | x_n, [\hat{y}_n], [y_n])$",
        "",
        "Testing:",
        r"$\hat{x}_t^o = \arg\max_{M_o} p(\hat{x}_t^o | x_t^n)$",
        "",
        "✅ No dependency on specific generator",
        "✅ Works with any downstream $M_g$"
    ]
    
    y_pos = 0.7
    for eq in equations_fipo:
        if eq in ["Training:", "Testing:"]:
            ax2.text(0.1, y_pos, eq, transform=ax2.transAxes, 
                    fontsize=12, weight='bold')
        elif eq.startswith("$"):
            ax2.text(0.5, y_pos, eq, transform=ax2.transAxes, ha='center', 
                    fontsize=14, fontfamily='serif')
        elif eq.startswith("✅"):
            ax2.text(0.5, y_pos, eq, transform=ax2.transAxes, ha='center', 
                    fontsize=12, color='green')
        y_pos -= 0.1
    
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

explain_mathematical_difference()

## 2. Cross-Model Performance Analysis

### 2.1 FIPO Performance Across Different Models

In [ ]:
@dataclass
class ModelPerformance:
    """Performance data for a model"""
    model_name: str
    model_size: str
    naive_scores: Dict[str, float]
    optimized_scores: Dict[str, float]
    
    def get_improvement(self, benchmark: str) -> float:
        """Calculate improvement percentage"""
        naive = self.naive_scores.get(benchmark, 0)
        optimized = self.optimized_scores.get(benchmark, 0)
        if naive == 0:
            return 0
        return ((optimized - naive) / naive) * 100

# Load performance data from Table 2
def load_fipo_results() -> List[ModelPerformance]:
    """Load FIPO results from paper"""
    
    models_data = [
        ModelPerformance(
            model_name="Llama2-7B",
            model_size="7B",
            naive_scores={"GSM8K": 8.89, "BBH": 31.21, "PiQA": 62.78, 
                         "CosmosQA": 43.09, "MMLU": 46.58},
            optimized_scores={"GSM8K": 11.70, "BBH": 33.50, "PiQA": 69.37, 
                            "CosmosQA": 52.11, "MMLU": 54.56}
        ),
        ModelPerformance(
            model_name="Tulu2-13B",
            model_size="13B",
            naive_scores={"GSM8K": 39.06, "BBH": 36.49, "PiQA": 76.62, 
                         "CosmosQA": 55.13, "MMLU": 57.43},
            optimized_scores={"GSM8K": 40.17, "BBH": 40.26, "PiQA": 78.58, 
                            "CosmosQA": 57.68, "MMLU": 59.10}
        ),
        ModelPerformance(
            model_name="Baichuan2-13B",
            model_size="13B",
            naive_scores={"GSM8K": 46.81, "BBH": 37.95, "PiQA": 68.56, 
                         "CosmosQA": 51.88, "MMLU": 57.46},
            optimized_scores={"GSM8K": 48.12, "BBH": 39.95, "PiQA": 74.77, 
                            "CosmosQA": 56.88, "MMLU": 58.32}
        )
    ]
    
    return models_data

def visualize_cross_model_performance():
    """Visualize FIPO performance across models"""
    
    models_data = load_fipo_results()
    benchmarks = ["GSM8K", "BBH", "PiQA", "CosmosQA", "MMLU"]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    # Plot for each benchmark
    for idx, benchmark in enumerate(benchmarks):
        ax = axes[idx]
        
        model_names = [m.model_name for m in models_data]
        naive_scores = [m.naive_scores[benchmark] for m in models_data]
        optimized_scores = [m.optimized_scores[benchmark] for m in models_data]
        improvements = [m.get_improvement(benchmark) for m in models_data]
        
        x = np.arange(len(model_names))
        width = 0.35
        
        bars1 = ax.bar(x - width/2, naive_scores, width, label='Naive', 
                       color='lightcoral', alpha=0.7)
        bars2 = ax.bar(x + width/2, optimized_scores, width, label='FIPO', 
                       color='lightgreen', alpha=0.7)
        
        # Add improvement percentages
        for i, (imp, opt_score) in enumerate(zip(improvements, optimized_scores)):
            ax.text(i, opt_score + 1, f'+{imp:.1f}%', ha='center', 
                   fontsize=9, color='darkgreen', weight='bold')
        
        ax.set_xlabel('Models')
        ax.set_ylabel('Score (%)')
        ax.set_title(f'{benchmark} Performance', fontsize=14, weight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(model_names, rotation=45)
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')
    
    # Overall improvement summary
    ax = axes[5]
    
    # Calculate average improvements
    avg_improvements = []
    for model in models_data:
        avg_imp = np.mean([model.get_improvement(b) for b in benchmarks])
        avg_improvements.append(avg_imp)
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']  # Different colors for each model
    bars = ax.bar(range(len(models_data)), avg_improvements, color=colors, alpha=0.7)
    
    # Add value labels
    for i, (bar, imp) in enumerate(zip(bars, avg_improvements)):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.2,
               f'{imp:.1f}%', ha='center', va='bottom', fontsize=12, weight='bold')
    
    ax.set_xlabel('Models')
    ax.set_ylabel('Average Improvement (%)')
    ax.set_title('Average Improvement Across All Benchmarks', fontsize=14, weight='bold')
    ax.set_xticks(range(len(models_data)))
    ax.set_xticklabels([m.model_name for m in models_data])
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.suptitle('FIPO Performance Across Different Models', fontsize=18, weight='bold')
    plt.tight_layout()
    plt.show()

visualize_cross_model_performance()

### 2.2 Extended Analysis with More Models

In [ ]:
def analyze_extended_models():
    """Analyze FIPO on extended model set from Figure 4"""
    
    # Data from Figure 4 (6 BBH tasks average)
    extended_results = pd.DataFrame({
        'Model': ['Llama2-7B', 'Tulu2-7B', 'Llama3-70B-Instruct', 
                 'Qwen2-72B-Instruct', 'GPT-3.5-turbo', 'GPT-4o'],
        'Size': ['7B', '7B', '70B', '72B', 'Unknown', 'Unknown'],
        'Type': ['Open', 'Open', 'Open', 'Open', 'Proprietary', 'Proprietary'],
        'Naive': [46.8, 44.7, 66.6, 68.0, 51.3, 76.2],
        'APE': [49.2, 48.9, 67.6, 68.7, 68.1, 79.7],
        'PromptAgent': [54.1, 52.4, 67.7, 72.2, 79.0, 82.0],
        'GPT-4': [53.1, 54.7, 66.5, 69.1, 68.0, 81.3],
        'FIPO': [56.7, 55.4, 69.2, 73.1, 73.2, 84.4]
    })
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # Left: Performance comparison
    methods = ['Naive', 'APE', 'PromptAgent', 'GPT-4', 'FIPO']
    colors = ['gray', 'lightblue', 'lightgreen', 'lightyellow', 'lightcoral']
    
    x = np.arange(len(extended_results))
    width = 0.15
    
    for i, (method, color) in enumerate(zip(methods, colors)):
        offset = (i - 2) * width
        bars = ax1.bar(x + offset, extended_results[method], width, 
                       label=method, color=color, alpha=0.8)
        
        # Highlight FIPO
        if method == 'FIPO':
            for bar in bars:
                bar.set_edgecolor('black')
                bar.set_linewidth(2)
    
    ax1.set_xlabel('Models', fontsize=12)
    ax1.set_ylabel('Performance (%)', fontsize=12)
    ax1.set_title('Performance Comparison Across Methods', fontsize=14, weight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(extended_results['Model'], rotation=45, ha='right')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Right: Improvement analysis
    improvements = pd.DataFrame()
    for method in methods[1:]:
        improvements[method] = ((extended_results[method] - extended_results['Naive']) / 
                               extended_results['Naive'] * 100)
    
    # Group by model type
    model_types = {
        '7B Models': ['Llama2-7B', 'Tulu2-7B'],
        '70B+ Models': ['Llama3-70B-Instruct', 'Qwen2-72B-Instruct'],
        'Proprietary': ['GPT-3.5-turbo', 'GPT-4o']
    }
    
    y_pos = 0
    y_labels = []
    
    for group_name, models in model_types.items():
        for model in models:
            if model in extended_results['Model'].values:
                idx = extended_results[extended_results['Model'] == model].index[0]
                
                # Plot improvements
                for j, method in enumerate(['APE', 'PromptAgent', 'GPT-4', 'FIPO']):
                    imp = improvements.loc[idx, method]
                    color = colors[j+1]
                    
                    bar = ax2.barh(y_pos, imp, height=0.2, 
                                  left=j*25, color=color, alpha=0.8)
                    
                    # Add value
                    ax2.text(j*25 + imp/2, y_pos, f'{imp:.1f}%', 
                            ha='center', va='center', fontsize=8)
                
                y_labels.append(model)
                y_pos += 1
        
        # Add group separator
        if group_name != 'Proprietary':
            ax2.axhline(y_pos - 0.5, color='gray', linestyle='--', alpha=0.5)
    
    ax2.set_yticks(range(len(y_labels)))
    ax2.set_yticklabels(y_labels)
    ax2.set_xlabel('Improvement over Naive (%)', fontsize=12)
    ax2.set_title('Improvement Analysis by Model Type', fontsize=14, weight='bold')
    ax2.grid(True, alpha=0.3, axis='x')
    
    # Add method labels
    for j, method in enumerate(['APE', 'PromptAgent', 'GPT-4', 'FIPO']):
        ax2.text(j*25 + 12.5, -0.8, method, ha='center', fontsize=10, weight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\n📊 Model-Agnostic Performance Summary:")
    print("=" * 50)
    
    for group_name, models in model_types.items():
        group_data = extended_results[extended_results['Model'].isin(models)]
        if not group_data.empty:
            avg_fipo_imp = ((group_data['FIPO'] - group_data['Naive']) / 
                           group_data['Naive'] * 100).mean()
            print(f"{group_name}: Average FIPO improvement = {avg_fipo_imp:.1f}%")
    
    print("\n🔍 Key Insights:")
    print("• FIPO consistently outperforms other methods")
    print("• Larger improvements on smaller models (7B)")
    print("• Works effectively on both open and proprietary models")

analyze_extended_models()

## 3. Model-Agnostic Design Principles

### 3.1 Key Design Elements

In [ ]:
class ModelAgnosticOptimizer:
    """Demonstrates model-agnostic design principles"""
    
    def __init__(self):
        self.design_principles = {
            "No In-Box Dependency": {
                "description": "Optimizer trained offline without specific generator",
                "benefit": "Works with any model at inference time",
                "implementation": "Use diverse training data, not model-specific"
            },
            "Universal Meta-Template": {
                "description": "Template adapts to presence/absence of responses",
                "benefit": "Flexible across training and inference",
                "implementation": "Optional components in template"
            },
            "Task-Focused Optimization": {
                "description": "Optimize task instruction, not model behavior",
                "benefit": "Generalizes across model architectures",
                "implementation": "Focus on clarity, structure, specificity"
            },
            "Dataset Diversification": {
                "description": "8 format types reduce model-specific bias",
                "benefit": "Robust to different model capabilities",
                "implementation": "2×2×2 diversification strategy"
            }
        }
    
    def visualize_principles(self):
        """Visualize design principles"""
        
        fig, ax = plt.subplots(figsize=(14, 10))
        
        # Create network graph
        G = nx.Graph()
        
        # Central node
        G.add_node("Model-Agnostic\nFIPO", size=3000, color='gold')
        
        # Principle nodes
        colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow']
        for i, (principle, details) in enumerate(self.design_principles.items()):
            G.add_node(principle, size=2000, color=colors[i])
            G.add_edge("Model-Agnostic\nFIPO", principle)
            
            # Add benefit nodes
            benefit_node = f"Benefit {i+1}"
            G.add_node(benefit_node, size=1000, color='lightgray')
            G.add_edge(principle, benefit_node)
        
        # Layout
        pos = nx.spring_layout(G, k=2, iterations=50)
        
        # Draw nodes
        node_colors = [G.nodes[node]['color'] for node in G.nodes()]
        node_sizes = [G.nodes[node].get('size', 1500) for node in G.nodes()]
        
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                              node_size=node_sizes, alpha=0.8)
        nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5, width=2)
        nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
        
        ax.set_title("Model-Agnostic Design Principles", fontsize=18, weight='bold', pad=20)
        ax.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Show detailed explanation
        print("\n🏗️ Model-Agnostic Design Principles:")
        print("=" * 60)
        
        for principle, details in self.design_principles.items():
            print(f"\n📌 {principle}")
            print(f"   Description: {details['description']}")
            print(f"   Benefit: {details['benefit']}")
            print(f"   Implementation: {details['implementation']}")

optimizer = ModelAgnosticOptimizer()
optimizer.visualize_principles()

### 3.2 Comparison with Model-Specific Approaches

In [ ]:
def compare_optimization_strategies():
    """Compare different optimization strategies"""
    
    strategies = pd.DataFrame({
        'Approach': ['APE', 'PromptAgent', 'Direct GPT-4', 'FIPO'],
        'Type': ['Model-Specific', 'Model-Specific', 'Model-Specific', 'Model-Agnostic'],
        'Training_Required': ['No', 'No', 'No', 'Yes'],
        'API_Dependency': ['Yes', 'Yes', 'Yes', 'No'],
        'Privacy_Safe': ['No', 'No', 'No', 'Yes'],
        'Generalization': ['Poor', 'Poor', 'Poor', 'Excellent'],
        'Cost_Per_Optimization': ['$5', '$5', '$4', '$0'],
        'Speed': ['2h', '2h', '1h', '30s']
    })
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    
    # Left: Feature comparison
    features = ['Training_Required', 'API_Dependency', 'Privacy_Safe']
    
    # Convert to binary for visualization
    binary_data = np.zeros((len(strategies), len(features)))
    for i, approach in enumerate(strategies['Approach']):
        for j, feature in enumerate(features):
            value = strategies.loc[i, feature]
            binary_data[i, j] = 1 if value == 'Yes' else 0
    
    im = ax1.imshow(binary_data, cmap='RdYlGn', aspect='auto')
    
    # Set ticks
    ax1.set_xticks(np.arange(len(features)))
    ax1.set_yticks(np.arange(len(strategies)))
    ax1.set_xticklabels(features, rotation=45, ha='right')
    ax1.set_yticklabels(strategies['Approach'])
    
    # Add text annotations
    for i in range(len(strategies)):
        for j in range(len(features)):
            text = ax1.text(j, i, '✓' if binary_data[i, j] else '✗',
                           ha='center', va='center', fontsize=16,
                           color='white' if binary_data[i, j] else 'black')
    
    ax1.set_title('Feature Comparison', fontsize=14, weight='bold')
    
    # Right: Performance vs Cost
    # Extract numeric values
    costs = [5, 5, 4, 0]  # Dollar amounts
    speeds = [120, 120, 60, 0.5]  # Minutes
    
    # Create bubble chart
    colors = ['red', 'orange', 'yellow', 'green']
    sizes = [100, 100, 100, 200]  # FIPO is larger
    
    for i, approach in enumerate(strategies['Approach']):
        ax2.scatter(costs[i], speeds[i], s=sizes[i]*5, 
                   c=[colors[i]], alpha=0.6, edgecolors='black', linewidth=2)
        ax2.annotate(approach, (costs[i], speeds[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=10)
    
    ax2.set_xlabel('Cost per Optimization ($)', fontsize=12)
    ax2.set_ylabel('Time per Optimization (minutes)', fontsize=12)
    ax2.set_title('Cost vs Speed Analysis', fontsize=14, weight='bold')
    ax2.set_yscale('log')
    ax2.grid(True, alpha=0.3)
    
    # Add ideal zone
    ideal_rect = Rectangle((0, 0), 1, 1, linewidth=2, 
                          edgecolor='green', facecolor='green', alpha=0.1)
    ax2.add_patch(ideal_rect)
    ax2.text(0.5, 0.5, 'Ideal\nZone', ha='center', va='center', 
            fontsize=12, color='green', weight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Summary table
    print("\n📊 Strategy Comparison Summary:")
    print(strategies.to_string(index=False))

compare_optimization_strategies()

## 4. Implementing Model-Agnostic Optimization

### 4.1 Cross-Model Testing Framework

In [ ]:
class CrossModelTester:
    """Framework for testing FIPO across different models"""
    
    def __init__(self):
        self.test_models = [
            {"name": "Small-7B", "size": 7, "type": "decoder"},
            {"name": "Medium-13B", "size": 13, "type": "decoder"},
            {"name": "Large-70B", "size": 70, "type": "decoder"},
            {"name": "Instruction-Tuned", "size": 13, "type": "instruction"},
            {"name": "Chat-Model", "size": 7, "type": "chat"},
            {"name": "Multilingual", "size": 13, "type": "multilingual"}
        ]
        
        self.test_prompts = [
            {"task": "Math", "naive": "Calculate 15% of 200"},
            {"task": "Reasoning", "naive": "Why does ice float?"},
            {"task": "Creative", "naive": "Write a haiku about AI"},
            {"task": "Factual", "naive": "What is quantum computing?"}
        ]
    
    def simulate_optimization(self, prompt: str, model_type: str) -> Tuple[str, float]:
        """Simulate FIPO optimization"""
        
        # Base optimization (model-agnostic)
        optimized = f"Please carefully and systematically {prompt.lower()}. "
        optimized += "Provide a clear, step-by-step explanation."
        
        # Simulate performance improvement
        base_improvement = np.random.uniform(5, 15)
        
        # Model-specific adjustments (but FIPO doesn't know this!)
        model_factors = {
            "decoder": 1.0,
            "instruction": 1.2,
            "chat": 1.1,
            "multilingual": 0.9
        }
        
        improvement = base_improvement * model_factors.get(model_type, 1.0)
        
        return optimized, improvement
    
    def run_cross_model_test(self):
        """Run optimization across all models"""
        
        results = []
        
        for model in self.test_models:
            for prompt_data in self.test_prompts:
                optimized, improvement = self.simulate_optimization(
                    prompt_data["naive"], model["type"]
                )
                
                results.append({
                    "model": model["name"],
                    "model_size": model["size"],
                    "model_type": model["type"],
                    "task": prompt_data["task"],
                    "improvement": improvement
                })
        
        return pd.DataFrame(results)
    
    def visualize_results(self, results_df: pd.DataFrame):
        """Visualize cross-model results"""
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # 1. Heatmap of improvements
        ax = axes[0, 0]
        pivot_data = results_df.pivot(index='model', columns='task', values='improvement')
        sns.heatmap(pivot_data, annot=True, fmt='.1f', cmap='YlOrRd', ax=ax)
        ax.set_title('Improvement Heatmap (Model × Task)', fontsize=14, weight='bold')
        
        # 2. Model size vs improvement
        ax = axes[0, 1]
        avg_by_size = results_df.groupby('model_size')['improvement'].mean()
        ax.plot(avg_by_size.index, avg_by_size.values, 'o-', linewidth=3, 
               markersize=10, color='darkblue')
        ax.set_xlabel('Model Size (B parameters)')
        ax.set_ylabel('Average Improvement (%)')
        ax.set_title('Model Size vs Improvement', fontsize=14, weight='bold')
        ax.grid(True, alpha=0.3)
        
        # 3. Task-wise performance
        ax = axes[1, 0]
        task_avg = results_df.groupby('task')['improvement'].agg(['mean', 'std'])
        x = range(len(task_avg))
        ax.bar(x, task_avg['mean'], yerr=task_avg['std'], 
              capsize=5, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
        ax.set_xticks(x)
        ax.set_xticklabels(task_avg.index)
        ax.set_ylabel('Average Improvement (%)')
        ax.set_title('Task-wise Performance', fontsize=14, weight='bold')
        ax.grid(True, alpha=0.3, axis='y')
        
        # 4. Model type analysis
        ax = axes[1, 1]
        type_avg = results_df.groupby('model_type')['improvement'].mean().sort_values()
        colors = plt.cm.viridis(np.linspace(0, 1, len(type_avg)))
        bars = ax.barh(range(len(type_avg)), type_avg.values, color=colors)
        
        # Add value labels
        for i, (bar, value) in enumerate(zip(bars, type_avg.values)):
            ax.text(value + 0.2, i, f'{value:.1f}%', va='center')
        
        ax.set_yticks(range(len(type_avg)))
        ax.set_yticklabels(type_avg.index)
        ax.set_xlabel('Average Improvement (%)')
        ax.set_title('Performance by Model Type', fontsize=14, weight='bold')
        ax.grid(True, alpha=0.3, axis='x')
        
        plt.suptitle('FIPO Cross-Model Testing Results', fontsize=18, weight='bold')
        plt.tight_layout()
        plt.show()

# Run cross-model testing
tester = CrossModelTester()
results = tester.run_cross_model_test()
tester.visualize_results(results)

print("\n🎯 Cross-Model Testing Summary:")
print(f"Total configurations tested: {len(results)}")
print(f"Average improvement: {results['improvement'].mean():.1f}%")
print(f"Std deviation: {results['improvement'].std():.1f}%")
print("\n✅ FIPO shows consistent improvement across all model types!")

## 5. Practical Guidelines

### 5.1 Best Practices for Model-Agnostic Optimization

In [ ]:
def create_best_practices_guide():
    """Create visual guide for best practices"""
    
    best_practices = {
        "Training Phase": [
            "Use diverse model outputs for training data",
            "Apply 8-type format diversification",
            "Train with both small and large model responses",
            "Include multilingual examples if possible"
        ],
        "Optimization Focus": [
            "Optimize clarity and structure, not model quirks",
            "Use universal improvement patterns",
            "Avoid model-specific terminology",
            "Focus on task understanding"
        ],
        "Testing Strategy": [
            "Test on models not seen during training",
            "Evaluate across different model sizes",
            "Include both base and instruction-tuned models",
            "Measure relative improvement, not absolute scores"
        ],
        "Deployment": [
            "Use same optimizer for all downstream models",
            "No need for model-specific fine-tuning",
            "Monitor performance across model updates",
            "Maintain single optimization pipeline"
        ]
    }
    
    fig, ax = plt.subplots(figsize=(14, 10))
    ax.axis('off')
    
    # Title
    ax.text(0.5, 0.95, "Model-Agnostic Optimization Best Practices", 
           ha='center', fontsize=20, weight='bold', transform=ax.transAxes)
    
    # Create quadrants
    colors = ['#FFE5E5', '#E5F5FF', '#E5FFE5', '#FFF5E5']
    positions = [(0.02, 0.5), (0.52, 0.5), (0.02, 0.05), (0.52, 0.05)]
    
    for i, (category, practices) in enumerate(best_practices.items()):
        x, y = positions[i]
        
        # Category box
        rect = FancyBboxPatch((x, y), 0.46, 0.4, 
                             boxstyle="round,pad=0.02",
                             facecolor=colors[i],
                             edgecolor='gray',
                             linewidth=2,
                             transform=ax.transAxes)
        ax.add_patch(rect)
        
        # Category title
        ax.text(x + 0.23, y + 0.35, category, 
               ha='center', fontsize=14, weight='bold',
               transform=ax.transAxes)
        
        # Practices
        for j, practice in enumerate(practices):
            ax.text(x + 0.03, y + 0.28 - j*0.07, f"• {practice}", 
                   fontsize=11, transform=ax.transAxes,
                   wrap=True)
    
    # Add central insight
    center_text = "Key Insight:\nFIPO optimizes prompts,\nnot model behavior"
    ax.text(0.5, 0.48, center_text, ha='center', va='center',
           fontsize=12, weight='bold', style='italic',
           bbox=dict(boxstyle="round,pad=0.5", facecolor='yellow', alpha=0.3),
           transform=ax.transAxes)
    
    plt.tight_layout()
    plt.show()

create_best_practices_guide()

# Implementation example
print("\n💻 Implementation Example:")
print("=" * 50)
print("""
# Model-agnostic FIPO usage

# 1. Load trained FIPO optimizer (same for all models)
fipo_optimizer = load_fipo_model("path/to/fipo_weights")

# 2. Optimize prompt (no model info needed)
naive_prompt = "Explain quantum computing"
optimized_prompt = fipo_optimizer.optimize(naive_prompt)

# 3. Use with ANY model
model_a_response = model_a.generate(optimized_prompt)
model_b_response = model_b.generate(optimized_prompt)
model_c_response = model_c.generate(optimized_prompt)

# All models benefit from the same optimization!
""")

## 6. Summary & Key Takeaways

### Core Model-Agnostic Concepts:

1. **Independence from Generators**:
   - No in-box testing required
   - Trained offline once, used everywhere
   - No API dependencies

2. **Universal Optimization**:
   - Focus on task clarity, not model quirks
   - Works across model sizes (7B to 70B+)
   - Effective for both open and proprietary models

3. **Performance Insights**:
   - Average 6.37% improvement on 7B models
   - Average 2.13% improvement on 13B models
   - Consistent gains across all benchmarks
   - Larger relative gains on smaller models

4. **Practical Benefits**:
   - Single optimization pipeline for all models
   - Privacy-preserving (no external APIs)
   - Cost-effective ($0 per optimization)
   - Fast inference (30 seconds vs hours)

### Key Success Factors:

- **Diverse Training Data**: Not tied to specific model outputs
- **Format Diversification**: 8 types reduce model bias
- **Task-Focused Design**: Optimize instructions, not behaviors
- **Offline Training**: Complete independence from test models

FIPO's model-agnostic design represents a paradigm shift from traditional prompt optimization!